In [119]:
# entry: unitary
# parameters in the unitary?

In [120]:
# from unitary to tensor
def Uni_to_ten(uni_matrix, phys_site_num, d_bond, d_phys_index, L):
    if phys_site_num == 0:
        res_matrix = uni_matrix[0:1, d_phys_index * d_bond: (d_phys_index+1)*d_bond]
    elif phys_site_num == L-1:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: d_phys_index * d_bond + 1]
    else:
        res_matrix = uni_matrix[0:d_bond, d_phys_index * d_bond: (d_phys_index + 1) * d_bond]
    return res_matrix

In [121]:
# from res_matrix to three-leg tensor:
import tenpy.linalg.np_conserved as npc
import numpy as np
def Site_Ten(uni_matrix, phys_site_num, phys_leg_num, bond_leg_num, L):
    if phys_site_num == 0:
        res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, phys_site_num, bond_leg_num, i, L) \
                                                 for i in range(0,phys_leg_num)],labels=['p0','vL','vR'])
    elif phys_site_num == L-1:
        res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, phys_site_num, bond_leg_num, i, L) \
                                                 for i in range(0,phys_leg_num)],labels=['p'+str(phys_site_num),'vL','vR'])
    else:
        res_tensor = npc.Array.from_ndarray_trivial([Uni_to_ten(uni_matrix, phys_site_num, bond_leg_num, i, L) \
                                                 for i in range(0,phys_leg_num)],labels=['p'+str(phys_site_num),'vL','vR'])
    return res_tensor

In [122]:
import tenpy
import numpy as np
import tenpy.linalg.np_conserved as npc
from scipy.stats import unitary_group
# for purpose of test, generate an arbitrary unitary matrix
x = unitary_group.rvs(4)
x
x11 = unitary_group.rvs(4)
# test Uni_to_ten
x1 = Uni_to_ten(x,0,2,1,2)
print(x1)
# test Site_ten
x_t = Site_Ten(x,0,2,2,2)
x_t1 = Site_Ten(x11,1,2,2,2)
print (x_t.rank)
#print(x_t)
x2 = npc.Array.to_ndarray(x_t)
x11_2 = npc.Array.to_ndarray(x_t1)
print(x2)
print(x11_2)

[[ 0.02632271-0.18775696j -0.24329483+0.5502272j ]]
3
[[[ 0.36160277-0.33710874j  0.46003517+0.38220527j]]

 [[ 0.02632271-0.18775696j -0.24329483+0.5502272j ]]]
[[[-0.4161125 +0.50563997j]
  [-0.38320732+0.37913321j]]

 [[ 0.44107565+0.26640638j]
  [-0.01372861-0.50351203j]]]


In [158]:
# test for MPS.from_full
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc
site = BosonSite()
print(site)
L = 2
lat = Chain(L, site, bc_MPS = 'finite')
print(lat)
#psi = MPS(lat.mps_sites(), state, bc='finte', form='B')
#psi = MPS.from_Bflat(lat.mps_sites(), state, bc='finte', form='B')
state = npc.tensordot(x_t, x_t1, ('vR','vL'))
print(type(state))
psi = MPS.from_full(lat.mps_sites(), state)
print(type(psi))
print(psi)

BosonSite(1, 'N', 0.000000)
<class 'tenpy.linalg.np_conserved.Array'>
<class 'tenpy.networks.mps.MPS'>
MPS, L=2, bc='finite'.
chi: [2]
sites: BosonSite(1, 'N', 0.000000) BosonSite(1, 'N', 0.000000)
forms: (1.0, 0.0) (0.0, 1.0)


In [159]:
# from tensor to MPS
from tenpy.networks.mps import MPS
from tenpy.networks.site import BosonSite
from tenpy.models.lattice import Chain
import tenpy.linalg.np_conserved as npc

def MPS_state(UM_list, d_phys, d_bond, L):
    site = BosonSite()
    lat = Chain(L, site, bc_MPS = 'finite')
    state = Site_Ten(UM_list[0], 0, d_phys, d_bond, L)
    # print(state)
    for i in range(1, L):
        state1 = Site_Ten(UM_list[i], i, d_phys, d_bond, L)
        # print(state1)
        state = npc.tensordot(state, state1,('vR','vL'))
    psi = MPS.from_full(lat.mps_sites(), state)
    # other methods to consider, MPS.from_Bflat, MPS
    # problem with MPS.from_Bflat now
    return psi

In [161]:
# test MPS_state function
print(MPS_state([x,x11], 2, 2, 2))

MPS, L=2, bc='finite'.
chi: [2]
sites: BosonSite(1, 'N', 0.000000) BosonSite(1, 'N', 0.000000)
forms: (1.0, 0.0) (0.0, 1.0)


In [126]:
# description of Hamiltonian as a MPO?


In [ ]:
from tenpy.networks.mpo import MPO, MPOEnvironment
from tenpy.networks.mps import MPS
# H_mpo is the mpo operator of H; 
# psi is in MPS format that constructed from the tensor
# L total number of sites
def MPO_H_contract(H_mpo, psi, L):
    env = MPOEnvironment(psi, H_mpo, psi)
    E = env.full_contraction(L-1)
    return E